# Prep 1: steps to install the packages

In [1]:
!pip install py_entitymatching
!pip install scipy
!pip install numpy
!pip install pandas

     |████████████████████████████████| 788kB 1.2MB/s eta 0:00:01
  ERROR: Complete output from command /usr/local/opt/python/bin/python3.7 -u -c 'import setuptools, tokenize;__file__='"'"'/private/var/folders/zk/8rp60fz5295fdzgznzrhrhhh0000gn/T/pip-install-d8d43lx4/py-stringsimjoin/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/zk/8rp60fz5295fdzgznzrhrhhh0000gn/T/pip-wheel-0x0mfqwt --python-tag cp37:
  ERROR: running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-10.14-x86_64-3.7
  creating build/lib.macosx-10.14-x86_64-3.7/py_stringsimjoin
  copying py_stringsimjoin/__init__.py -> build/lib.macosx-10.14-x86_64-3.7/py_stringsimjoin
  creating build/lib.macosx-10.14-x86_64-3.7/py_stringsimjoin/join
  copying py_stringsimjoin/join/overlap_join.py -> build/lib.macosx-10.14-x86_64-3.

Failed to build py-stringsimjoin
  Running setup.py install for py-stringsimjoin ... error
    ERROR: Complete output from command /usr/local/opt/python/bin/python3.7 -u -c 'import setuptools, tokenize;__file__='"'"'/private/var/folders/zk/8rp60fz5295fdzgznzrhrhhh0000gn/T/pip-install-d8d43lx4/py-stringsimjoin/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /private/var/folders/zk/8rp60fz5295fdzgznzrhrhhh0000gn/T/pip-record-esi1ptly/install-record.txt --single-version-externally-managed --compile:
    ERROR: running install
    running build
    running build_py
    running egg_info
    writing py_stringsimjoin.egg-info/PKG-INFO
    writing dependency_links to py_stringsimjoin.egg-info/dependency_links.txt
    writing requirements to py_stringsimjoin.egg-info/requires.txt
    writing top-level names to py_stringsimjoin.egg-info/top_level.txt
   

# [NEED MODIFICATION: Modify this cell to point to the file location]
# Prep 2: enter the file location on your harddisk

In [4]:
table_a = '<table_a_location_downloaded_from_cloudmatcher_loc>'
table_b = '<table_a_location_downloaded_from_cloudmatcher_loc>'
candidate_set = '<candidate_set_downloaded_from_cloudmatcher_loc>'
prediction_set = '<prediction_set_downloaded_from_cloudmatcher_loc>'

# Prep 3: reading the files into pandas dataframe

In [3]:
dfa = pd.read_csv(table_a)
dfb = pd.read_csv(table_b)
dfc = pd.read_csv(candidate_set)
dfp = pd.read_csv(prediction_set)

# Module: debug_blocker
# Description: debug the blocking rule using the below script to ensure you are not dropping true matches
# Note: You need to run Prep 1 and 2 in order to run this

In [35]:
# Example input format:
# Format of table_a:
# _id, attribute1, attribute2, ....., attributen

# Format of table_b:
# _id, attribute1, attribute2, ....., attributen

# Format of candidate_set
# A_id,B_id
# where A_id is _id from table_a and B_id is the _id column value from table_b

In [4]:
import py_entitymatching as em
import pandas as pd

def run_debug_blocker(table_a, table_b, table_a_key, table_b_key, candidate_set):
    dfl = em.read_csv_metadata(table_a, key=table_a_key)
    dfr = em.read_csv_metadata(table_b, key=table_b_key)

    # reading the candidate set and adding key
    dfcand = pd.read_csv(candidate_set)
    dfcand.drop_duplicates(inplace=True)
    dfcand.to_csv('cand_set_with_index.csv', index_label='id')

    dfcset = em.read_csv_metadata('cand_set_with_index.csv', key='id', ltable=dfl, 
                                  rtable=dfr, fk_ltable='A_id', fk_rtable='B_id')

    # running debug blocker to identify the records in A x B \ C
    debug_file = em.debug_blocker(dfcset, dfl, dfr)
    
    return debug_file

In [12]:
debug_file = run_debug_blocker(table_a, table_b, '_id', '_id', candidate_set)

In [ ]:
debug_file

# Module: estimate_precision_recall
# Description: the below code helps you get an estimation of P/R on the candidate set.

In [8]:
import pandas as pd
from scipy.stats import norm
from numpy import sqrt

delta = .05
Z = norm.ppf(1 - (delta / 2))

def estimate_PR(labeled_pairs, reduced_cands, predicted_matches):
    '''
    labeled_pairs - a pandas dataframe with schema id1,id2,label
                    Note label needs to be Boolean

    reduced_cands - a pandas dataframe with schema id1,id2
    predicted_matches - a pandas dataframe with schema id1,id2
    
    return:
        ( (recall lower bound, recall upper bound), (precision lower bound, precision upper bound) )
    '''

    labeled_pairs.drop_duplicates(inplace=True)
    labeled_pairs.columns = ['id1', 'id2', 'label']
    reduced_cands.columns = ['id1', 'id2']
    reduced_cand_set = set(zip(reduced_cands.id1, reduced_cands.id2))
    predicted_matches = set(zip(predicted_matches.id1, predicted_matches.id2))
    
    # estimate the recall
    # number of positives in the labeled sample
    actual_pos = float(labeled_pairs.label.sum())
    # the maximum number of postives in the candidate set
    max_actual_pos = float(actual_pos + len(reduced_cand_set) - len(labeled_pairs))
    
    # true positives in the labeled sample
    true_pos = float(labeled_pairs.apply(lambda x : (x['id1'], x['id2']) in predicted_matches and x['label'], axis=1).sum())
    #estimated recall
    recall = float(true_pos / actual_pos)

    recall_error = Z * sqrt( ((recall * (1 - recall)) / (actual_pos)) * ((max_actual_pos - actual_pos) / (max_actual_pos - 1)) )


    # estimate Precision
    labeled_set  = set(zip(labeled_pairs.id1, labeled_pairs.id2))
    predicted_pos = float(len(labeled_set & predicted_matches))
    
    predicted_pos_in_reduced_cand_set = float(len(reduced_cand_set & predicted_matches))
    
    alpha =  predicted_pos_in_reduced_cand_set / len(predicted_matches)
    precision = alpha * (true_pos / predicted_pos)
    
    precision_error = alpha * Z * sqrt( ((precision * (1 - precision)) / predicted_pos) * (float((len(predicted_matches) - predicted_pos)) / (len(predicted_matches)  - 1)) )

    return ((recall - recall_error, recall + recall_error),
            (precision - precision_error, precision + precision_error))

# [NEED MODIFICATION: Modify this cell to point to the file location]

In [11]:
# read the labeled pairs file, i.e. the file with the labels
labeled_pairs = pd.read_csv('labeled_pairs.csv')
print(estimate_PR(labeled_pairs, dfc, dfp))

((0.9825476990384129, 0.9981596964599794), (1.0, 1.0))


# Alternatively if you run into issues running this script on your laptop you can use
# https://colab.research.google.com/notebooks/welcome.ipynb